<a href="https://colab.research.google.com/github/AlphaAtlas/Diffusion-Compilaton-Testing/blob/main/Hidet_Debug_Repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Check GPU
!nvidia-smi

Wed Jun  7 17:45:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Upgrade numpy. This will require a runtime restart
!pip install --upgrade numpy


In [2]:
#Install requirements, including the git version of diffusers with graph break fixes
!pip install --pre transformers accelerate
#!pip install torch torchvision pytorch-triton --extra-index-url https://download.pytorch.org/whl/nightly/cu121 --pre --upgrade  #Torch nightly is not needed to reproduce the bugs
!pip install -v -U git+https://github.com/huggingface/diffusers.git
!pip install --pre --extra-index-url https://download.hidet.org/whl hidet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.hidet.org/whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.6/663.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.4/428.4 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 64.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0rc1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0rc1 which is incompatible.


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-cbo9gezp
  Running command git version
  git version 2.25.1
  Running command git clone --filter=blob:none https://github.com/huggingface/diffusers.git /tmp/pip-req-build-cbo9gezp
  Cloning into '/tmp/pip-req-build-cbo9gezp'...
  Updating files:   0% (2/752)
  Updating files:   1% (8/752)
  Updating files:   2% (16/752)
  Updating files:   3% (23/752)
  Updating files:   4% (31/752)
  Updating files:   5% (38/752)
  Updating files:   6% (46/752)
  Updating files:   7% (53/752)
  Updating files:   8% (61/752)
  Updating files:   9% (68/752)
  Updating files:  10% (76/752)
  Updating files:  11% (83/752)
  Updating files:  12% (91/752)
  Updating files:  13% (98/752)
  Updating files:  14% (106/752)
  Updating files:  15% (113/752)
  Up

In [1]:
#Check torch version
import torch
print(torch.__version__)

2.0.1+cu118


In [ ]:
import hidet
import diffusers
from diffusers import StableDiffusionPipeline
import torch
import torch._dynamo


#Create pipeline
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe = pipe.to("cuda")
#Make sure the unet is contiguous
pipe.unet = pipe.unet.to(memory_format=torch.contiguous_format)


#Some typical diffusers pipeline optimizations
# pipe.unet.to(memory_format=torch.channels_last) #Unsupported by hidet, but does not seem to make a difference if disabled.
#pipe.enable_vae_tiling()
#pipe.enable_xformers_memory_efficient_attention()
#
#print("### Post optimization benchmark:")
image = pipe("a photo of an astronaut riding a horse on mars").images[0]


hidet.option.parallel_build(True)
torch._dynamo.config.suppress_errors = True
torch._dynamo.disallow_in_graph(diffusers.models.attention.BasicTransformerBlock)

# more search
hidet.torch.dynamo_config.search_space(0)
#hidet.torch.dynamo_config.dump_graph_ir("./local_graph")
hidet.option.cache_dir("local_cache")
# automatically transform the model to use float16 data type
hidet.torch.dynamo_config.use_fp16(True)
# use float16 data type as the accumulate data type in operators with reduction
hidet.torch.dynamo_config.use_fp16_reduction(True)
# use tensorcore
hidet.torch.dynamo_config.use_tensor_core()
pipe.unet = torch.compile(pipe.unet, backend="hidet")

print("### torch.compile warmup:")
image = pipe("a photo of an astronaut riding a horse on mars").images[0]

print("torch.compile benchmark:")
image = pipe("a photo of an astronaut riding a horse on mars").images[0]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


  0%|          | 0/50 [00:00<?, ?it/s]

### torch.compile warmup:


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:hidet.graph.frontend.torch.dynamo_backends:received a subgraph with 751 nodes to optimize
DEBUG:hidet.graph.frontend.torch.dynamo_backends:graph: graph():
    %sample : torch.Tensor [#users=1] = placeholder[target=sample]
    %timestep : torch.Tensor [#users=1] = placeholder[target=timestep]
    %encoder_hidden_states : torch.Tensor [#users=16] = placeholder[target=encoder_hidden_states]
    %getitem : [#users=1] = call_function[target=operator.getitem](args = (%timestep, None), kwargs = {})
    %to : [#users=1] = call_method[target=to](args = (%getitem, cuda:0), kwargs = {})
    %expand : [#users=1] = call_method[target=expand](args = (%to, 2), kwargs = {})
    %arange : [#users=1] = call_function[target=torch.arange](args = (), kwargs = {start: 0, end: 160, dtype: torch.float32, device: cuda:0})
    %mul : [#users=1] = call_function[target=operator.mul](args = (-9.210340371976184, %arange), kwargs = {})
    %truediv : [#users=1] = call_function[target=operator.truediv](args = (%